<a href="https://colab.research.google.com/github/LauraOrozco/extraccion_texto/blob/main/text_extract_azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 41 kB 526 kB/s 


In [ ]:
!pip install pillow

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "SUBSCRIPTION_KEY"
endpoint = "ENDPOINT"

In [ ]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [ ]:
'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_path = os.path.join ("./", "17-12-2021 lote 2.jpg")
# Open the image
read_image = open(read_image_path, "rb")


===== Read File - remote =====


In [ ]:
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

In [ ]:
# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    with open('17-12-2021 lote 2.txt', 'w') as f:
      for text_result in read_result.analyze_result.read_results:
          for line in text_result.lines:
              f.write(line.text)
              f.write('\n')


In [1]:
import glob
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

#!ls "/gdrive/My Drive/folder"

files = glob.glob(f"/gdrive/My Drive/data360/17_dic/*.jpg")

Mounted at /gdrive


In [2]:
path = "/gdrive/My Drive/data360/17_dic/"
file_names = [x.replace(path,"") for x in files]

In [3]:
def api_text_extract(path: str, file_name: str):
  read_image_path = os.path.join (path, file_name)
  read_image = open(read_image_path, "rb")
  read_response = computervision_client.read_in_stream(read_image, raw=True)
  read_operation_location = read_response.headers["Operation-Location"]
  operation_id = read_operation_location.split("/")[-1]
  while True:
      read_result = computervision_client.get_read_result(operation_id)
      if read_result.status.lower () not in ['notstarted', 'running']:
          break
      print ('Waiting for result...')
      time.sleep(10)

  if read_result.status == OperationStatusCodes.succeeded:
      with open(path+file_name+'.txt', 'w') as f:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                f.write(line.text)
                f.write('\n')

In [ ]:
for image in file_names:
  api_text_extract(path, image)


Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...
Waiting for result...


In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [5]:
import pandas as pd
from scipy import ndimage

In [6]:
from PIL import Image

In [7]:
def getSkewAngle(cvImage) -> float:
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle

In [8]:
def limpieza_imagen(imagen):
    gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (1, 1), 0)
    ret,thresh = cv2.threshold(blur, 127,255,cv2.THRESH_TOZERO)
    kernel = np.ones((3,3), np.uint8)
    img_erosion = cv2.erode(thresh, kernel, iterations=1)
    rotated = ndimage.rotate(img_erosion, -1*getSkewAngle(img))
    return rotated

In [9]:
file_names

['Leonardo Jiménez 17-12-2021(5).jpg',
 'Leonardo Jiménez 17-12-2021(1).jpg',
 'Leonardo Jiménez 17-12-2021(4).jpg',
 'Leonardo Jiménez 17-12-2021(2).jpg',
 'Leonardo Jiménez 17-12-2021.jpg',
 'Leonardo Jiménez 17-12-2021(9).jpg',
 'Leonardo Jiménez 17-12-2021(7).jpg',
 'Leonardo Jiménez 17-12-2021(6).jpg',
 'Leonardo Jiménez 17-12-2021(8).jpg',
 '17-12-2021 lote 2(2).jpg',
 '17-12-2021 lote 2.jpg',
 '17-12-2021 lote 2(1).jpg',
 '17-12-2021 lote 2(3).jpg',
 '17-12-2021 lote 2(4).jpg',
 '17-12-2021 lote 2(5).jpg',
 'Leonardo Jiménez 17-12-2021(10).jpg',
 'Leonardo Jiménez 17-12-2021(3).jpg']

In [10]:
for i in range(len(file_names)):
    img = cv2.imread(path+"/"+file_names[i])
    img_pross = limpieza_imagen(img)
    im = Image.fromarray(img_pross)
    im.save(path+"/"+file_names[i][:-4] + "_pross.jpg")
